#Transcrição dos áudios

Bibliotecas e Funcionalidades iniciais para transcrição:

Whisper: É um modelo de transcrição de áudio desenvolvido pela OpenAI. Ele é utilizado para converter áudio em texto, suportando múltiplos idiomas e podendo ser aplicado em tarefas de reconhecimento de fala (speech-to-text).

PyTorch: É uma biblioteca de machine learning de código aberto amplamente utilizada para desenvolvimento e treinamento de modelos de inteligência artificial. Ela é conhecida pela sua facilidade de uso e flexibilidade, permitindo a criação de redes neurais complexas de forma eficiente.


CUDA (Compute Unified Device Architecture) é uma plataforma de computação paralela e uma interface de programação de aplicativos (API) criada pela NVIDIA. Ela permite que desenvolvedores utilizem o poder de processamento de GPUs (unidades de processamento gráfico) para realizar cálculos e tarefas computacionais que não estão restritas ao processamento gráfico tradicional.

Essas bibliotecas funcionam de forma integrada, e o PyTorch com suporte a CUDA permite o uso de GPUs para acelerar o treinamento e a inferência de modelos, o que é especialmente útil em tarefas que envolvem grandes quantidades de dados, como transcrição de áudio com Whisper.


In [ ]:
# Instalar as bibliotecas necessárias
!pip install git+https://github.com/openai/whisper.git
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install requests

# Importar as bibliotecas necessárias
import whisper
import torch
import requests
import os

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-gr075ogj
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-gr075ogj
  Resolved https://github.com/openai/whisper.git to commit 279133e3107392276dc509148da1f41bfb532c7e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
# Verificar se o ambiente está utilizando GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Dispositivo em uso: {device}")

# Carregar o modelo 'medium' no dispositivo selecionado
model = whisper.load_model('medium', device=device)

# Função para obter a lista de arquivos de áudio do repositório GitHub
def get_audio_files_from_github(repo_owner, repo_name, folder_path=''):
    url = f'https://api.github.com/repos/{repo_owner}/{repo_name}/contents/{folder_path}'
    response = requests.get(url)
    if response.status_code == 200:
        contents = response.json()
        audio_files = [file['name'] for file in contents if file['type'] == 'file' and file['name'].lower().endswith(('.mp3', '.wav', '.m4a', '.flac', '.aac', '.ogg'))]
        return audio_files
    else:
        print(f"Erro ao acessar o repositório GitHub: {response.status_code}")
        return []

# Função para baixar o arquivo de áudio do GitHub
def download_file_from_github(repo_owner, repo_name, file_path, local_filename):
    url = f'https://raw.githubusercontent.com/{repo_owner}/{repo_name}/main/{file_path}'
    response = requests.get(url)
    if response.status_code == 200:
        with open(local_filename, 'wb') as f:
            f.write(response.content)
        return True
    else:
        print(f"Erro ao baixar o arquivo: {response.status_code}")
        return False

# Informações do repositório GitHub
repo_owner = 'cesaroliveiragoes'
repo_name = 'Audios_Sprint_03'

while True:
    # Listar os arquivos de áudio disponíveis no GitHub
    print("\nArquivos de áudio disponíveis no GitHub:")
    audio_files = get_audio_files_from_github(repo_owner, repo_name)
    if not audio_files:
        print("Nenhum arquivo de áudio encontrado.")
        break
    for filename in audio_files:
        print(filename)

    # Solicitar o nome do arquivo de áudio
    audio_file = input("Digite o nome do arquivo de áudio para transcrever: ")

    if audio_file not in audio_files:
        print("Arquivo não encontrado no repositório GitHub.")
        continue

    try:
        # Baixar o arquivo de áudio do GitHub
        if download_file_from_github(repo_owner, repo_name, audio_file, audio_file):
            # Transcrever o arquivo de áudio
            result = model.transcribe(audio_file, fp16=torch.cuda.is_available())

            # Exibir o texto transcrito
            print("\nTexto transcrito:")
            print(result['text'])

            # Salvar a transcrição em um arquivo .txt
            txt_file = audio_file.rsplit('.', 1)[0] + '.txt'
            with open(txt_file, 'w', encoding='utf-8') as f:
                f.write(result['text'])
            print(f"\nTranscrição salva em: {txt_file}")

            # Remover o arquivo de áudio temporário
            os.remove(audio_file)

    except Exception as e:
        print(f"Ocorreu um erro: {e}")

    # Perguntar se o usuário deseja transcrever outro arquivo
    continuar = input("\nDeseja transcrever outro arquivo? (sim/não): ").strip().lower()
    if continuar != 'sim':
        print("Encerrando o programa.")
        break


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-w32lh2fc
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-w32lh2fc
  Resolved https://github.com/openai/whisper.git to commit 279133e3107392276dc509148da1f41bfb532c7e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 63.5 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=802819 sha256=fa4d47f92d2347064df879df3fc26fa7ffe2d38bf5120c2c883f08f6d5772aa4
  Stored in directory: /tmp/pip-ephem-wheel-cache-fxunomhh/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
Looking in indexes: https://download.pytorch.org/whl/cu118
Dispositivo em uso:

100%|█████████████████████████████████████| 1.42G/1.42G [00:24<00:00, 63.5MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper


Arquivos de áudio disponíveis no GitHub:
2874830.wav
2962046.wav
2962184.wav
2962219.wav
2962417.wav
2962972.wav
2963930.wav
2964126.wav
2964909.wav
2965023.wav
2967070.wav
Digite o nome do arquivo de áudio para transcrever: 2874830.wav

Texto transcrito:
 Oi, bom dia, falo com o Sr. Pedro? Bom dia, sou eu mesmo. Sr. Pedro, me chamo Beatriz, falo da TOTS, tudo bem? Da TOTS? Isso. Certo. Tudo bem, Sr. Pedro? Tudo bom, é? Que bom. O Sr. seria o responsável ainda pelo DP e utiliza o sistema na empresa Complexo Brasil? Sim, o responsável pelo DP é a Ana Carla. O Sr. seria pelo RH? Porque pra mim aqui tá como um DP, o Sr. É, mas o responsável no DP é a Ana Carla. Ah, entendi. O Sr. não seria responsável então? Não, eu trabalho com o sistema, sim, ajudo, faz as foras de pagamento, mas a coordenadora é a Ana Carla. Ah, entendi. É que eu preciso fazer um acompanhamento com o responsável pra gente saber como que tá aí a experiência de vocês. Será que ela tem dois minutinhos pra falar comigo ag

# Análise de Sentimento para Classificação

In [ ]:
import nltk # Importar a biblioteca NLTK (Natural Language Toolkit) em Python.
from sklearn.model_selection import train_test_split # Essa função é usada para dividir um conjunto de dados em subconjuntos de treino e teste de forma aleatória
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB # classificadores
from sklearn.metrics import accuracy_score # calcula a precisão (acurácia) de um modelo de classificação
import pickle # serializar e salvar objetos Python em arquivos e também para carregar esses objetos de volta para a memória
from nltk.corpus import stopwords # Da biblioteca nltk.corpus, ela contém uma lista de palavras irrelevantes (como "o", "a", "e") que são frequentemente removidas
from nltk.stem import SnowballStemmer # Da biblioteca nltk, o stemmer é usado para reduzir palavras ao seu radical, removendo sufixos ou prefixos
from sklearn.feature_extraction.text import CountVectorizer # Essa classe converte um conjunto de documentos em uma matriz de bag-of-words, essencial para transformar texto em dados numéricos para ML
import pandas as pd
import numpy as np
import re

In [ ]:
# carregar dataset público com série de frases e sentimentos do twitter já calssificados para treinar o modelo
# disponível https://www.kaggle.com/datasets/gazprom/anlise-de-sentimentos-pt-br
data = pd.read_csv("df_analise_sentimento.csv", encoding='latin1') # adiciona o parâmetro encoding e define como latin1
print(data.shape)
data.head()

(2996, 2)


,Sentence,Sentiment
0,Os pedidos de produtos de processo são para a ...,neutral
1,Grupo Intertek anuncia atualização estratégica...,neutral
2,Barclays contrata o novo diretor de operações ...,neutral
3,A divisão de private equity do FL Group admini...,neutral
4,Pode ser qualquer um de nÃ³s a qualquer moment...,neutral


In [ ]:
data.info() # exibir um resumo sobre o DataFrame

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2996 entries, 0 to 2995
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Sentence   2996 non-null   object
 1   Sentiment  2996 non-null   object
dtypes: object(2)
memory usage: 46.9+ KB


In [ ]:
data.Sentiment.value_counts() # contar a frequência de cada valor único na coluna Sentiment

,count
Sentiment,
neutral,1000
positive,999
negative,997


In [ ]:
data.Sentence[0]

'Os pedidos de produtos de processo são para a instalação de equipamentos de filtração de separação em três projetos de gasoduto de gás natural na China, América do Sul e Arábia Saudita.'

In [ ]:
# Função será usada para limpar o texto fornecido, removendo partes indesejadas
def limpar(texto):
  limpo = re.compile(r'<.*?>') #  Criar um padrão de correspondência. O padrão r'<.*?>' corresponde a qualquer texto que esteja dentro de tags HTML,
  #ou seja, algo que comece com <, tenha qualquer conteúdo entre os símbolos e termine com >.
  return re.sub(limpo,'',texto) # Retornar o texto limpo de qualquer um desses símbolos de HTML

data.Sentence = data.Sentence.apply(limpar)
data.Sentence[0]

'Os pedidos de produtos de processo são para a instalação de equipamentos de filtração de separação em três projetos de gasoduto de gás natural na China, América do Sul e Arábia Saudita.'

In [ ]:
# Função para remover caracteres especiais
def especial(texto):
  rem = '' #  variável rem como uma string vazia
  for i in texto: # loop for que percorre cada caractere individual (i) no texto
    if i.isalnum(): #  Retorna True se o caractere for uma letra ou um número
      rem = rem + i # Se o caractere for alfanumérico (letra ou número), ele é adicionado à string rem
    else:
      rem = rem + ' '
  return rem

data.Sentence = data.Sentence.apply(especial)
data.Sentence[0]

'Os pedidos de produtos de processo são para a instalação de equipamentos de filtração de separação em três projetos de gasoduto de gás natural na China  América do Sul e Arábia Saudita '

In [ ]:
# Converter pra minúsculas
def minusculo(texto):
  return texto.lower()

data.Sentence = data.Sentence.apply(lambda x: x.lower())
data.Sentence[0]

'os pedidos de produtos de processo são para a instalação de equipamentos de filtração de separação em três projetos de gasoduto de gás natural na china  américa do sul e arábia saudita '

In [ ]:
# Limpeza do dataset usando o nltk
nltk.download('stopwords') # Faz o download da lista de palavras de stopwords do NLTK, que são palavras comuns como 'de', 'a', 'o' que geralmente não agregam valor
nltk.download('punkt') # Faz o download de dados para a função de tokenização de palavras, que divide frases em palavras individuais
from nltk.tokenize import word_tokenize # Importa a função de tokenização que quebra o texto em palavras individuais (tokens)

def rem_stopwords(texto):
  stop_words = set(stopwords.words('portuguese')) # Obtém a lista de stopwords em português e as armazena em um conjunto
  words = word_tokenize(texto) # Usa o word_tokenize para quebrar o texto em uma lista de palavras (tokens)
  return [w for w in words if w not in stop_words] # Retorna uma lista de palavras filtrada, removendo as stopwords.

data.Sentence = data.Sentence.apply(rem_stopwords)
data.Sentence[0]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['pedidos',
 'produtos',
 'processo',
 'instalação',
 'equipamentos',
 'filtração',
 'separação',
 'três',
 'projetos',
 'gasoduto',
 'gás',
 'natural',
 'china',
 'américa',
 'sul',
 'arábia',
 'saudita']

In [ ]:
def stem_txt(texto):
  ss = SnowballStemmer('portuguese') # Cria um objeto stemmer para a língua portuguesa usando o algoritmo Snowball Stemmer
  return ' '.join([ss.stem(palavra) for palavra in texto]) # Aplica o stemming em cada palavra do texto e une-as de volta em uma string

data.Sentence = data.Sentence.apply(stem_txt)
data.Sentence[0]

'ped produt process instal equip filtraçã separ três projet gasodut gás natural chin amér sul aráb saudit'

In [ ]:
X = np.array(data.iloc[:,0].values) # Extrai todos os valores da primeira coluna do DataFrame data usando o .iloc
y = np.array(data.Sentiment.values) # Extrai os valores da coluna Sentiment do DataFrame data, que contém os rótulos que tentaremos prever
cv = CountVectorizer(max_features=1000) # Ferramenta do sklearn usada para converter uma coleção de documentos de texto em uma matriz de Bag of Words
X = cv.fit_transform(data.Sentence).toarray() # Aplica a transformação CountVectorizer na coluna Sentence do DataFrame
print('X.shape = ', X.shape)
print('y.shape = ', y.shape)

X.shape =  (2996, 1000)
y.shape =  (2996,)


In [ ]:
print(X)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
# Treinar test split
# Dividir os dados em conjuntos de treino e teste
trainx, textx, trainy, testy = train_test_split(X, y, test_size=0.2, random_state=9)
print('Train shapes : X = {}, y = {}' .format(trainx.shape, trainy.shape)) #  Exibe as dimensões do conjunto de treino
print('Test shapes : X = {}, y = {}' .format(textx.shape, testy.shape)) # # Exibe as dimensões do conjunto de teste

Train shapes : X = (2396, 1000), y = (2396,)
Test shapes : X = (600, 1000), y = (600,)


Criando o Modelo - Bag of Words

In [ ]:
# Definir os modelos e treiná-los
# Modelo Naive Bayes Gaussiano, adequado para dados contínuos e assume que as características seguem uma distribuição normal.
# Modelo Naive Bayes Multinomial, adequado para dados categóricos e texto
# Modelo Naive Bayes Bernoulli, adequado para dados binários ou booleanos;
gnb, mnb, bnb = GaussianNB(), MultinomialNB(alpha=1.0, fit_prior=True), BernoulliNB(alpha=1.0, fit_prior=True)
gnb.fit(trainx, trainy) # Treina o modelo GaussianNB com o conjunto de treino (features e rótulos)
mnb.fit(trainx, trainy) # Treina o modelo MultinomialNB com o conjunto de treino
bnb.fit(trainx, trainy) # Treina o modelo BernoulliNB com o conjunto de treino

BernoulliNB()

In [ ]:
# Predição e acurácia para escolher o melhor modelo
# Realiza previsões no conjunto de teste usando cada um dos modelos treinados
ypg = gnb.predict(textx)
ypm = mnb.predict(textx)
ypb = bnb.predict(textx)

In [ ]:
# Mostrar qual possui mairo acurácia, quanto mais próximo de 1 melhor
print('Guassiana = ', accuracy_score(testy, ypg))
print('Multinomial = ', accuracy_score(testy, ypm))
print('Bernoulli = ', accuracy_score(testy, ypb))

Guassiana =  0.55
Multinomial =  0.6433333333333333
Bernoulli =  0.65


In [ ]:
pickle.dump(bnb, open('model.pkl', 'wb'))

In [ ]:
def ler_arquivo():
    nome_arquivo = input("Digite o nome do arquivo .txt: ")
    rev = ""  # Inicializa a variável rev como uma string vazia

    try:
        with open(nome_arquivo, 'r') as arquivo:
            rev = arquivo.read()
    except FileNotFoundError:
        print(f"O arquivo '{nome_arquivo}' não foi encontrado.")
    except Exception as e:
        print(f"Ocorreu um erro ao ler o arquivo: {e}")

    return rev

# Chamando a função e utilizando o valor retornado
conteudo_do_arquivo = ler_arquivo()
print("O conteúdo do arquivo é:", conteudo_do_arquivo)


Digite o nome do arquivo .txt: 2874830.txt
O conteúdo do arquivo é:  Oi, bom dia, falo com o Sr. Pedro? Bom dia, sou eu mesmo. Sr. Pedro, me chamo Beatriz, falo da TOTS, tudo bem? Da TOTS? Isso. Certo. Tudo bem, Sr. Pedro? Tudo bom, é? Que bom. O Sr. seria o responsável ainda pelo DP e utiliza o sistema na empresa Complexo Brasil? Sim, o responsável pelo DP é a Ana Carla. O Sr. seria pelo RH? Porque pra mim aqui tá como um DP, o Sr. É, mas o responsável no DP é a Ana Carla. Ah, entendi. O Sr. não seria responsável então? Não, eu trabalho com o sistema, sim, ajudo, faz as foras de pagamento, mas a coordenadora é a Ana Carla. Ah, entendi. É que eu preciso fazer um acompanhamento com o responsável pra gente saber como que tá aí a experiência de vocês. Será que ela tem dois minutinhos pra falar comigo agora? É a Ana Carla, né? É a Ana Carla. Tá bom. A Ana Carla da TOTS. Vou transmitir pra ela, 47-06. Obrigada. Ok. R.H. Ana Carla. Bom dia, Ana Carla. Beatriz da TOTS falando, tudo bem? Bom d

In [ ]:
# Aplicar as funções de pré-processamento no conteúdo do arquivo
f1 = limpar(conteudo_do_arquivo) # Remove as tags HTML do texto usando a função 'limpar' criada anteriormente
f2 = especial(f1) # Remove caracteres especiais, mantendo apenas caracteres alfanuméricos e espaços.
f3 = minusculo(f2) # Converte o texto para minúsculas
f4 = rem_stopwords(f3) # Remove as stopwords (palavras comuns que não agregam muito significado) do texto
f5 = stem_txt(f4) # Aplica stemming, reduzindo palavras às suas raízes

# Criar a bolsa de palavras (bag of words) a partir do texto pré-processado
bow, words = [], word_tokenize(f5) # Tokeniza o texto
for word in words:
  bow.append(words.count(word)) # Conta a frequência de cada palavra e adiciona ao 'bow'.

# Salvar o dicionário de palavras (vocabulario) em um arquivo usando pickle
word_dict = cv.vocabulary_ # Obtém o dicionário de palavras
pickle.dump(word_dict, open('bow.pk1', 'wb')) # Serializa o dicionário de palavras e salva em 'bow.pk1'

inp = [] # Preparar o vetor de entrada para o modelo
for i in word_dict:
  inp.append(f5.count(i[0])) # Conta a ocorrência de cada palavra do dicionário no texto pré-processado

# Fazer a previsão com o modelo treinado
y_pred = bnb.predict(np.array(inp).reshape(1, -1)) # Usa o modelo BernoulliNB para prever, pois tinha melhor acurácia
# Use reshape(1, -1) para calcular automaticamente o número de colunas baseadas no array inputado
# Printar o resultado final da previsão de sentimentos
print("O arquivo escolhido para análise de sentimento apresenta um sentimento:", y_pred)

O arquivo escolhido para análise de sentimento apresenta um sentimento: ['neutral']


# Extração das Notas para um Dataset

In [ ]:
# Instalar as bibliotecas necessárias
!pip install transformers

A biblioteca Transformers é uma poderosa e popular ferramenta de código aberto desenvolvida pela empresa Hugging Face. Ela fornece acesso a modelos de aprendizado profundo de última geração baseados na arquitetura Transformer, que são amplamente usados em várias tarefas de processamento de linguagem natural (NLP)

In [ ]:
# Importar a biblioteca
from transformers import pipeline

In [ ]:
# Função ajustada para extrair notas via Regular Expressions
def extrair_notas_apos_interrogacao_ajustada(transcricao):
    """
    Função para extrair a primeira nota (número entre 0 e 10) que aparece após uma interrogação "?",
    ou números seguidos de um ponto final "." que aparecem isolados após uma frase,
    ou que seguem o termo "nota".
    Ignora números seguidos diretamente de uma interrogação "?",
    bem como números irrelevantes como CNPJ, expressões "0 a 10", "0 a 10?", "0 a 10," e números
    no formato "01", "02", ..., "09", ou números maiores que 10 como 27.
    Também ignora números que aparecem antes da palavra "pergunta" ou "perguntas", e ignora "6 meses" ou qualquer número seguido de "meses".
    Considera apenas um número se ele aparecer repetido consecutivamente, como em "9. 9.".
    """
    # Remover expressões "0 a 10", "0 a 10?", "0 a 10," e "CNPJ" e números irrelevantes (números longos com mais de 2 dígitos)
    transcricao_limpa = re.sub(r"0\s*a\s*10[\?,\s]?", "", transcricao)
    transcricao_limpa = re.sub(r"CNPJ|[0-9]{3,}", "", transcricao_limpa)

    # Remover números seguidos por "?" (ex.: "9?") e remover números maiores que 10 (preservando o "10")
    transcricao_limpa = re.sub(r'\b\d+\?\b', '', transcricao_limpa)
    transcricao_limpa = re.sub(r'\b(?!10\b)\d{2,}\b', '', transcricao_limpa)  # Remove números maiores que 10, mas preserva 10

    # Remover números no formato "01", "02", ..., "09"
    transcricao_limpa = re.sub(r'\b0[1-9]\b', '', transcricao_limpa)

    # Remover números que aparecem antes das palavras "pergunta" ou "perguntas"
    transcricao_limpa = re.sub(r'\b\d+\s*(pergunta|perguntas)\b', '', transcricao_limpa, flags=re.IGNORECASE)

    # Remover números seguidos da palavra "meses" (ex.: "6 meses")
    transcricao_limpa = re.sub(r'\b\d+\s*meses\b', '', transcricao_limpa, flags=re.IGNORECASE)

    # Capturar números seguidos por ponto final após uma frase (ex.: "O atendimento deles. 9. 9.")
    notas_com_ponto_final = re.findall(r'\.\s*(\d+)\.', transcricao_limpa)

    # Capturar números que vêm após o termo "nota " (ex.: "nota 7")
    notas_apos_termo_nota = re.findall(r'nota\s*(\d+)', transcricao_limpa)

    # Incluir números seguidos de ponto (ex.: "7.") e tratá-los como notas válidas
    transcricao_limpa = re.sub(r'(\b\d+)\.', r'\1', transcricao_limpa)

    # Dividir a transcrição em partes com base na presença de "?"
    partes = re.split(r'\?', transcricao_limpa)

    notas = []

    # Iterar sobre as partes após cada interrogação
    for parte in partes[1:]:  # Ignorar o que vem antes da primeira "?"
        # Procurar o primeiro número após a interrogação
        match = re.search(r'\b\d+\b', parte)
        if match:
            nota = int(match.group())
            if 0 <= nota <= 10 and (not notas or notas[-1] != nota):  # Apenas números entre 0 e 10, sem duplicatas consecutivas
                notas.append(nota)

    # Adicionar as notas capturadas no padrão "O atendimento deles. 9. 9." e remover duplicatas consecutivas
    for i, nota in enumerate(notas_com_ponto_final):
        if i == 0 or int(nota) != int(notas_com_ponto_final[i - 1]):  # Evitar duplicatas consecutivas
            nota = int(nota)
            if 0 <= nota <= 10:
                notas.append(nota)

    # Adicionar as notas capturadas após o termo "nota"
    for nota in notas_apos_termo_nota:
        nota = int(nota)
        if 0 <= nota <= 10 and (not notas or notas[-1] != nota):  # Evitar duplicatas consecutivas
            notas.append(nota)

    return notas

# Inicializar uma lista para armazenar todas as listas de notas extraídas
todas_as_listas_de_notas = []

while True:
    # Pedir o nome do arquivo .txt como input do usuário
    nome_arquivo = input("Digite o nome do arquivo .txt (com extensão): ")

    # Ler o conteúdo do arquivo fornecido pelo usuário
    try:
        with open(nome_arquivo, 'r', encoding='utf-8') as f:
            nova_transcricao = f.read()

        # Processamento da transcrição com a função ajustada
        notas_ajustadas_nova_transcricao = extrair_notas_apos_interrogacao_ajustada(nova_transcricao)

        # Adicionar as notas extraídas à lista de todas as listas de notas
        todas_as_listas_de_notas.append(notas_ajustadas_nova_transcricao)

        # Calcular a média das notas para este arquivo
        if notas_ajustadas_nova_transcricao:
            media_nps = np.mean(notas_ajustadas_nova_transcricao)
        else:
            media_nps = np.nan  # Ou você pode definir como 0 ou outro valor padrão

        # Imprimir as notas e a média para este arquivo
        print(f"\nlista_notas = {notas_ajustadas_nova_transcricao}")
        print(f"media_nps = {media_nps}\n")

        # Perguntar se o usuário deseja processar outro arquivo
        continuar = input("Deseja processar outro arquivo? (sim/não): ").strip().lower()
        if continuar != 'sim':
            break

    except FileNotFoundError:
        print(f"O arquivo '{nome_arquivo}' não foi encontrado. Verifique o nome e tente novamente.")

# Encontrar o número máximo de notas entre todas as listas
max_len = max(len(lista) for lista in todas_as_listas_de_notas) if todas_as_listas_de_notas else 0

if max_len > 0:
    # Preencher as listas menores com valores NaN para que todas as listas tenham o mesmo comprimento
    listas_alinhadas = [lista + [np.nan] * (max_len - len(lista)) for lista in todas_as_listas_de_notas]

    # Criar um DataFrame onde cada coluna será uma "nota 1", "nota 2", etc.
    df = pd.DataFrame(listas_alinhadas, columns=[f"nota {i+1}" for i in range(max_len)])

    # Calcular a média das notas para cada linha, ignorando NaN
    df['media_nps'] = df.mean(axis=1, skipna=True)

    # Exibir o DataFrame resultante
    print("DataFrame com todas as notas e médias:")
    print(df)
else:
    print("Nenhuma nota foi extraída dos arquivos fornecidos.")


Digite o nome do arquivo .txt (com extensão): 2874830
O arquivo '2874830' não foi encontrado. Verifique o nome e tente novamente.
Digite o nome do arquivo .txt (com extensão): 2874830.txt

lista_notas = [10, 6, 9, 7]
media_nps = 8.0

Deseja processar outro arquivo? (sim/não): sim
Digite o nome do arquivo .txt (com extensão): 2962046.txt

lista_notas = [8, 6, 7, 6, 8, 7, 6]
media_nps = 6.857142857142857

Deseja processar outro arquivo? (sim/não): não
DataFrame com todas as notas e médias:
   nota 1  nota 2  nota 3  nota 4  nota 5  nota 6  nota 7  media_nps
0      10       6       9       7     NaN     NaN     NaN   8.000000
1       8       6       7       6     8.0     7.0     6.0   6.857143


In [ ]:
df.head(5)

,nota 1,nota 2,nota 3,nota 4,nota 5,nota 6,nota 7,media_nps
0,10,6,9,7,NaN,NaN,NaN,8.000000
1,8,6,7,6,8.0,7.0,6.0,6.857143


In [ ]:
# Salvar o DataFrame como 'dataset_nps.csv'
df.to_csv('dataset_nps.csv', index=False)
print("\nO DataFrame foi salvo como 'dataset_nps.csv'.")


O DataFrame foi salvo como 'dataset_nps.csv'.
